In [ ]:
from google.colab import drive
import numpy as np
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
#added to be able to read from google drive
drive.mount('/content/drive')
dataset = []

for person in range(1, 41):
    for img in range(1, 11):
        #edited to be able to read from google drive
        folder_path = '/content/drive/My Drive/dataset/s' + str(person) + '/'

        image = Image.open(folder_path + str(img) + '.pgm')
        vec = np.array(image).reshape(10304)

        dataset.append(vec)

D = np.array([example for example in dataset])
D = D.reshape((400, 10304))

#Genrating the label vector
labels =[]

for i in range(1, 41):
    labels.append(i * np.ones((10, 1)))
y = np.stack(labels, axis=0).reshape(400, 1)

#odd rows for training
x_train = D[::2]
y_train = y[::2]

#even rows for test
x_test = D[1::2]
y_test = y[1::2]

print(y_train[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[1.]


In [ ]:
from numpy import linalg

alphas = [0.80, 0.85, 0.90, 0.95]
Ks = [1,3,5,7]
mean = np.mean(x_train, axis=0)
centered_data = x_train - mean
covariance = (centered_data.T.dot(centered_data))/len(x_train)

In [ ]:

pca_evalues, pca_evectors = linalg.eigh(covariance)
x = pca_evalues.argsort()[::-1]
pca_evalues = pca_evalues[x]
pca_evectors = pca_evectors[:,x]


In [ ]:
#calculating number of eigen values needed
def count_eigenV(pca_evalues, alpha):
    total = sum(pca_evalues)
    current_sum = 0.0
    counter = 0
    while current_sum/total < alpha:
        current_sum += pca_evalues[counter]
        counter += 1
    return counter

#projecting train and test on projection matrix
def project(PJmat, train_data, test_data):
    PJmat = PJmat.transpose()
    prj_train = np.dot(train_data, PJmat)
    prj_test = np.dot(test_data, PJmat)
    return prj_train, prj_test

#k-neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
def Kneighb(k,train,test,testLabel,aplha,y_train):
    model = KNeighborsClassifier(n_neighbors=k)
    FitModel = model.fit(train, y_train)
    predictions = model.predict(test)
    accuracy = round(accuracy_score(testLabel, predicitions), 2)


    print("For alpha = ",aplha, "and k = ",k, "Accuracy = ",accuracy)
    return accuracy


In [ ]:
count_80 = count_eigenV(pca_evalues, 0.80)
count_85 = count_eigenV(pca_evalues, 0.85)
count_90 = count_eigenV(pca_evalues, 0.90)
count_95 = count_eigenV(pca_evalues, 0.95)

print(count_80)
print(count_85)
print(count_90)
print(count_95)

36
51
76
115


In [ ]:
#clac projection matrix for each alpha

prj_mat80 = np.vstack(pca_evectors.T[0:count_80])
prj_mat85 = np.vstack(pca_evectors.T[0:count_85])
prj_mat90 = np.vstack(pca_evectors.T[0:count_90])
prj_mat95 = np.vstack(pca_evectors.T[0:count_95])

PJtrain80, PJtest80 = project(prj_mat80, x_train, x_test)
PJtrain85, PJtest85 = project(prj_mat85, x_train, x_test)
PJtrain90, PJtest90 = project(prj_mat90, x_train, x_test)
PJtrain95, PJtest95 = project(prj_mat95, x_train, x_test)

#converting label from nd array to 1d array *needed in k-nearest*
y_train = y_train.ravel()
y_test = y_test.ravel()



In [ ]:
#converting all labels from float to int alson needed in k nearest 
y_train = list(map(int, y_train))
y_test = list(map(int, y_test))

#have fun!
i = 0
for i in range(4):
  x = Kneighb(Ks[i],PJtrain95,PJtest95,y_test,0.80,y_train)
  print("")
  x = Kneighb(Ks[i],PJtrain95,PJtest95,y_test,0.85,y_train)
  print("")
  x = Kneighb(Ks[i],PJtrain95,PJtest95,y_test,0.90,y_train)
  print("")
  x = Kneighb(Ks[i],PJtrain95,PJtest95,y_test,0.95,y_train)
  print("")

For alpha =  0.8 and k =  1 Accuracy =  94.0

For alpha =  0.85 and k =  1 Accuracy =  94.0

For alpha =  0.9 and k =  1 Accuracy =  94.0

For alpha =  0.95 and k =  1 Accuracy =  94.0

For alpha =  0.8 and k =  3 Accuracy =  91.5

For alpha =  0.85 and k =  3 Accuracy =  91.5

For alpha =  0.9 and k =  3 Accuracy =  91.5

For alpha =  0.95 and k =  3 Accuracy =  91.5

For alpha =  0.8 and k =  5 Accuracy =  85.0

For alpha =  0.85 and k =  5 Accuracy =  85.0

For alpha =  0.9 and k =  5 Accuracy =  85.0

For alpha =  0.95 and k =  5 Accuracy =  85.0

For alpha =  0.8 and k =  7 Accuracy =  74.0

For alpha =  0.85 and k =  7 Accuracy =  74.0

For alpha =  0.9 and k =  7 Accuracy =  74.0

For alpha =  0.95 and k =  7 Accuracy =  74.0

